## Requirements

1. Install pytest and pytest-cov library


In [0]:
#!pipenv install pytest pytest-cov

## Current flow:

1. For a python code it generates the unit tests using `pytest` library. The dashboard supports tests execution along with a coverage report. If the unit tests are fine, there is an option to save them for future use. It can happen, especially with Ollama , the tests are having a typing error. In this case the code can be edited in the right window and executed afterwards. 

2. Supports 3 models: 

- gpt-4o-mini
- claude-3-5-sonnet-20240620
- llama3.2

It is recommended though to use other models except Ollama, my tests showed the code returned from ollama required more supervision and editing. Some generated unit tests from ollama don't provide full coverage, but still it is a good starting point for building such a tool.

In [0]:
# imports
import re
import os
import sys
import textwrap
from dotenv import load_dotenv
from openai import OpenAI
import anthropic
import gradio as gr
from pathlib import Path
import subprocess
from IPython.display import Markdown

In [0]:
# Initialization

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
OPENAI_MODEL = "gpt-4o-mini"
CLAUDE_MODEL = "claude-3-5-sonnet-20240620"
openai = OpenAI()
claude = anthropic.Anthropic()

In [0]:
OLLAMA_API = "http://localhost:11434/api/chat"
HEADERS = {"Content-Type": "application/json"}
OLLAMA_MODEL = "llama3.2"

## Code execution

In [0]:

def extract_code(text):
    # Regular expression to find text between ``python and ``
    match = re.search(r"```python(.*?)```", text, re.DOTALL)

    if match:
        code = match.group(0).strip()  # Extract and strip extra spaces
    else:
        code = ""
        print("No matching substring found.")

    return code.replace("```python\n", "").replace("```", "")


def execute_coverage_report(python_interpreter=sys.executable):
    if not python_interpreter:
        raise EnvironmentError("Python interpreter not found in the specified virtual environment.")
    
    command = ["coverage", "run", "-m", "pytest"]

    try:
        result = subprocess.run(command, check=True, capture_output=True, text=True)
        print("Tests ran successfully!")
        print(result.stdout)
        return result.stdout
    except subprocess.CalledProcessError as e:
        print("Some tests failed!")
        print("Output:\n", e.stdout)
        print("Errors:\n", e.stderr)
        # Extracting failed test information
        return e.stdout

def save_unit_tests(code):

    match = re.search(r"def\s+(\w+)\(", code, re.DOTALL)

    if match:
        function_name = match.group(1).strip()  # Extract and strip extra spaces
    else:
        function_name = ""
        print("No matching substring found.")

    test_code_path = Path("tests")
    (test_code_path / f"test_{function_name}.py").write_text(extract_code(code))
    Path("tests", "test_code.py").unlink()
    

def execute_tests_in_venv(code_to_test, tests, python_interpreter=sys.executable):
    """
    Execute the given Python code string within the specified virtual environment.
    
    Args:
    - code_str: str, the Python code to execute.
    - venv_dir: str, the directory path to the virtual environment created by pipenv.
    """
    
    if not python_interpreter:
        raise EnvironmentError("Python interpreter not found in the specified virtual environment.")

    # Prepare the command to execute the code
    code_str = textwrap.dedent(code_to_test) + "\n" + extract_code(tests)
    test_code_path = Path("tests")
    test_code_path.mkdir(parents=True, exist_ok=True)
    (test_code_path / f"test_code.py").write_text(code_str)
    command = ["pytest", str(test_code_path)]

    try:
        result = subprocess.run(command, check=True, capture_output=True, text=True)
        print("Tests ran successfully!")
        print(result.stderr)
        return result.stdout
    except subprocess.CalledProcessError as e:
        print("Some tests failed!")
        print("Output:\n", e.stdout)
        print("Errors:\n", e.stderr)
        # Extracting failed test information
        failed_tests = []
        for line in e.stdout.splitlines():
            if "FAILED" in line and "::" in line:
                failed_tests.append(line.strip())
        if failed_tests:
            print("Failed Tests:")
            for test in failed_tests:
                print(test)
    
        return e.stdout
    

## Prompts and calls to the models

In [0]:
system_message = """You are a helpful assistant which helps developers to write unit test cases for their code."""

In [0]:
def get_user_prompt(code):

    user_prompt = """Test include:

    - Valid inputs with expected results.
    - Inputs that test the boundaries or limits of the function's behavior.
    - Invalid inputs or scenarios where the function is expected to raise exceptions.

    Structure:

    - Begin with all necessary imports. 
    - Do not create custom imports. 
    - Do not insert in the response the function for the tests.
    - Ensure proper error handling for tests that expect exceptions.
    - Clearly name the test functions to indicate their purpose (e.g., test_function_name).

    Example Structure:

    - Use pytest.raises to validate exceptions.
    - Use assertions to verify correct outputs for successful and edge cases.

    Documentation:

    - Add docstrings explaining what each test verifies."""
    user_prompt += code

    return user_prompt

In [0]:
def stream_gpt(code):

    user_prompt = get_user_prompt(code)
    stream = openai.chat.completions.create(
        model=OPENAI_MODEL,
        messages=[
            {"role": "system", "content": system_message},
            {
                "role": "user",
                "content": user_prompt,
            },
        ],
        stream=True,
    )

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        yield response
    
    return response

def stream_ollama(code):

    user_prompt = get_user_prompt(code)
    ollama_via_openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')
    stream = ollama_via_openai.chat.completions.create(
        model=OLLAMA_MODEL,
        messages=[
            {"role": "system", "content": system_message},
            {
                "role": "user",
                "content": user_prompt,
            },
        ],
        stream=True,
    )

    response = ""
    for chunk in stream:
        response += chunk.choices[0].delta.content or ""
        yield response
    
    return response


def stream_claude(code):
    user_prompt = get_user_prompt(code)
    result = claude.messages.stream(
        model=CLAUDE_MODEL,
        max_tokens=2000,
        system=system_message,
        messages=[
            {
                "role": "user",
                "content": user_prompt,
            }
        ],
    )
    reply = ""
    with result as stream:
        for text in stream.text_stream:
            reply += text
            yield reply
            print(text, end="", flush=True)
    return reply

## Code examples to test the inteface

In [0]:
function_to_test = """
    def lengthOfLongestSubstring(s):
        if not isinstance(s, str):
            raise TypeError("Input must be a string")
        max_length = 0
        substring = ""
        start_idx = 0
        while start_idx < len(s):
            string = s[start_idx:]
            for i, x in enumerate(string):
                substring += x
                if len(substring) == len(set((list(substring)))):
                    
                    if len(set((list(substring)))) > max_length:
                        
                        max_length = len(substring)

            start_idx += 1
            substring = ""
                  
                
        return max_length"""

In [0]:
test_code = """```python
import pytest

# Unit tests using pytest
def test_lengthOfLongestSubstring():
    assert lengthOfLongestSubstring("abcabcbb") == 3  # Case with repeating characters
    assert lengthOfLongestSubstring("bbbbb") == 1    # Case with all same characters
    assert lengthOfLongestSubstring("pwwkew") == 3    # Case with mixed characters
    assert lengthOfLongestSubstring("") == 0           # Empty string case
    assert lengthOfLongestSubstring("abcdef") == 6     # All unique characters
    assert lengthOfLongestSubstring("abca") == 3       # Case with pattern and repeat
    assert lengthOfLongestSubstring("dvdf") == 3       # Case with repeated characters separated
    assert lengthOfLongestSubstring("a") == 1           # Case with single character
    assert lengthOfLongestSubstring("au") == 2          # Case with unique two characters
```"""

In [0]:
def optimize(code, model):
    if model == "GPT":
        result = stream_gpt(code)
    elif model == "Claude":
        result = stream_claude(code)
    elif model == "Ollama":
        result = stream_ollama(code)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far
    return result

## Gradio interface

In [0]:
with gr.Blocks() as ui:
    gr.Markdown("## Write unit tests for Python code")
    with gr.Row():
        with gr.Column(scale=1, min_width=300):
            python = gr.Textbox(label="Python code:", value=function_to_test, lines=10)
            model = gr.Dropdown(["GPT", "Claude", "Ollama"], label="Select model", value="GPT")
            unit_tests = gr.Button("Write unit tests")
        with gr.Column(scale=1, min_width=300):
            unit_tests_out = gr.TextArea(label="Unit tests", value=test_code, elem_classes=["python"])
            unit_tests_run = gr.Button("Run unit tests")
            coverage_run = gr.Button("Coverage report")
            save_test_run = gr.Button("Save unit tests")
    with gr.Row():
        
        python_out = gr.TextArea(label="Unit tests result", elem_classes=["python"])
        coverage_out = gr.TextArea(label="Coverage report", elem_classes=["python"])
        

    unit_tests.click(optimize, inputs=[python, model], outputs=[unit_tests_out])
    unit_tests_run.click(execute_tests_in_venv, inputs=[python, unit_tests_out], outputs=[python_out])
    coverage_run.click(execute_coverage_report, outputs=[coverage_out])
    save_test_run.click(save_unit_tests, inputs=[unit_tests_out])


ui.launch(inbrowser=True)
# ui.launch()